In [ ]:
# Import Necessary Libraries
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import xrspatial as xs
from datashader import transfer_functions as tf
from datashader import utils as ds_utils
from datashader.colors import Greys9, inferno
from scipy.spatial import KDTree
from shapely import ops

import py3dep
import pynhd

# Define the bounding box for the region of interest
bbox = (-91.3817, 43.3235, -91.1209, 43.7881)

# Check available DEM resolutions for the specified bounding box
dem_res = py3dep.check_3dep_availability(bbox)
print(f"Available DEM resolutions: {dem_res}")

# Download DEM data at a resolution of 10 meters
resolution = 10
dem = py3dep.get_dem(bbox, resolution)

# Plot the DEM
fig, ax = plt.subplots(figsize=(8, 6), dpi=100)
dem.plot(ax=ax, robust=True)
plt.title("Digital Elevation Model (DEM)")
plt.show()

# Retrieve NHD flowline data within the bounding box
wd = pynhd.WaterData("nhdflowline_network")
flowlines = wd.bybox(bbox)

# Prepare flowline data, removing isolated segments
flowlines = pynhd.prepare_nhdplus(flowlines, 0, 0, 0, remove_isolated=True)
flowlines = flowlines[flowlines.levelpathi == flowlines.levelpathi.min()].copy()

# Plot flowlines over the DEM
fig, ax = plt.subplots(figsize=(10, 8), dpi=100)
flowlines.plot(ax=ax, color="red", label="Flowlines")
dem.plot(ax=ax, robust=True)
plt.title("Flowlines Over DEM")
plt.legend()
plt.show()

# Merge flowlines into a single line geometry and extract the elevation profile
lines = ops.linemerge(flowlines.geometry.tolist())
river_dem = py3dep.elevation_profile(lines, resolution, crs=flowlines.crs)

# Plot the river elevation profile
river_dem.plot(x="distance", figsize=(7, 3.2))
plt.title("River Elevation Profile")
plt.show()

# Define a function to interpolate DEM using Inverse Distance Weighting (IDW)
def idw(river_dem: xr.DataArray, dem: xr.DataArray, n_neighbors: int) -> xr.DataArray:
    """
    Interpolate grid DEM from river DEM using Inverse Distance Weighting (IDW).

    Parameters:
        river_dem (xr.DataArray): DEM along the river.
        dem (xr.DataArray): Grid DEM to interpolate.
        n_neighbors (int): Number of nearest neighbors.

    Returns:
        xr.DataArray: Interpolated DEM.
    """
    river_coords = np.column_stack((river_dem.x, river_dem.y))
    kdt = KDTree(river_coords)

    dem_grid = np.dstack(np.meshgrid(dem.x, dem.y)).reshape(-1, 2)
    distances, indices = kdt.query(dem_grid, k=n_neighbors)

    weights = np.reciprocal(distances)
    weights /= weights.sum(axis=1, keepdims=True)

    interpolated = np.sum(weights * river_dem.to_numpy()[indices], axis=1)
    interpolated = interpolated.reshape((dem.sizes["y"], dem.sizes["x"]))
    return xr.DataArray(interpolated, dims=("y", "x"), coords={"x": dem.x, "y": dem.y})

# Perform IDW interpolation
elevation = idw(river_dem, dem, n_neighbors=3)

# Calculate Relative Elevation Model (REM)
rem = dem - elevation

# Plot the interpolated elevation and flowlines
fig, ax = plt.subplots(figsize=(10, 8), dpi=100)
elevation.plot(ax=ax, cmap="viridis", robust=True)
flowlines.plot(ax=ax, color="red", label="Flowlines")
plt.title("Interpolated Elevation with Flowlines")
plt.legend()
plt.show()

# Create a hillshade from the DEM
hillshade = xs.hillshade(dem, angle_altitude=10, azimuth=90)

# Stack shaded visualizations
tf.Image.border = 0
img = tf.stack(
    tf.shade(dem, cmap=Greys9, how="linear"),
    tf.shade(hillshade, cmap=["black", "white"], how="linear", alpha=180),
    tf.shade(rem, cmap=inferno[::-1], span=[0, 7], how="log", alpha=200),
)

# Export the resulting image
output_path = Path("_static", "REM.png")
output_path.parent.mkdir(parents=True, exist_ok=True)
ds_utils.export_image(img[::-1], output_path.as_posix())
print(f"Image exported to: {output_path}")
